In [5]:
import librosa

In [6]:
import soundfile as sf

In [7]:
from scipy.signal import stft

In [8]:
from scipy.signal import resample

In [9]:
import scipy.io.wavfile as wavfile

In [10]:
import numpy as np
import scipy.signal as signal
import soundfile as sf
import matplotlib.pyplot as plt

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, BatchNormalization
from tensorflow.keras.activations import tanh, relu, sigmoid
import numpy as np


In [12]:
input_file = 'babble_8KHz.wav'
output_file = 'resampled_16KHz.wav'
target_sr = 16000

In [13]:

# Load the audio file
audio, sr = librosa.load(input_file, sr=None)

# Resample the audio to the target sample rate
resampled_audio = librosa.resample(audio,orig_sr= sr, target_sr=target_sr)

# Save the resampled audio to a new file
# librosa.output.write_wav(output_file,y= resampled_audio, sr=target_sr)
sf.write(output_file, resampled_audio, target_sr)

In [14]:
babble

NameError: name 'babble' is not defined

In [ ]:
babble, fs_babble = sf.read("resampled_16KHz.wav")

In [ ]:
babble.shape

In [20]:
# Initialization
Fs = 16000  # Sample frequency (Hz)
c = 340  # Sound velocity (m/s)
K = 256  # FFT length
M = 2  # Number of sensors
d = 0.2  # Inter sensor distance (m)
type_nf = 'spherical'  # Type of noise field: 'spherical' or 'cylindrical'
L = 14400  # Data length

In [15]:
data, Fs_data= sf.read("resampled_16KHz.wav")

In [16]:
data=data[:14400]

In [17]:
data.shape

(14400,)

In [18]:
Fs_data

16000

In [21]:
if Fs != Fs_data:
    raise ValueError('Sample frequency of input file is incorrect.')
data = data - np.mean(data)
babble = np.zeros((L, M))
for m in range(M):
    babble[:, m] = data[:]

In [22]:
# Generate matrix with desired spatial coherence
ww = Fs * np.arange(K // 2 + 1) / K
DC = np.zeros((M, M, K // 2 + 1))
for p in range(M):
    for q in range(M):
        if p == q:
            DC[p, q, :] = np.ones((1, 1, K // 2 + 1))
        else:
            if type_nf == 'spherical':
                DC[p, q, :] = np.sinc(ww * np.abs(p - q) * d / (c * np.pi))
            elif type_nf == 'cylindrical':
                DC[p, q, :] = signal.jn(0, ww * np.abs(p - q) * d / c)
            else:
                raise ValueError('Unknown noise field.')

In [23]:
len(DC[0][0])

129

In [24]:
# Compare desired and generated coherence
K_eval = 256
ww = 2 * np.pi * Fs * np.arange(K_eval // 2 + 1) / K_eval
sc_theory = np.zeros((M - 1, K_eval // 2 + 1))
sc_generated = np.zeros((M - 1, K_eval // 2 + 1))

In [25]:
# Generate sensor signals with desired spatial coherence
def mix_signals(babble, DC, method):
    L, M = babble.shape
    x = np.zeros((L, M-1))
    for k in range(K // 2 + 1):
        if method == 'cholesky':
            mat_t=np.array(DC[:,:,k])
            C = linalg.cholesky(mat_t)
        elif method == 'eigenvalue':
            eigenvalues, eigenvectors = np.linalg.eig(DC[:, :, k])
            eigenvalues = np.maximum(eigenvalues, 0)
            C = eigenvectors @ np.diag(np.sqrt(eigenvalues)) @ eigenvectors.T
        else:
            raise ValueError('Unknown method.')
        x_k = babble @ C.T
        x[:, :M-1] = x_k[:, :M-1]
    return x

In [26]:
x = mix_signals(babble, DC, 'eigenvalue')

In [27]:
x

array([[-0.02627196],
       [-0.04455214],
       [-0.05170172],
       ...,
       [ 0.00322005],
       [-0.00170882],
       [-0.00114011]])

In [28]:
x.shape

(14400, 1)

In [29]:
len(x)

14400

In [30]:
sum=0
xx=np.zeros(L)
xx[:]=x[:,0]
# for a in x:
#     xx
#     if(a[1]>0):
#         print("fds")
#         sum+=a[1]
print(sum)

0


In [31]:
xx.shape

(14400,)

In [32]:
window_length = 256
frame_shift = 128
padding_length = window_length - (14400 % window_length)
padded_xx = np.pad(xx, (0, padding_length), 'constant')
frequencies, time, X=stft(padded_xx, fs=16000, window='hann', nperseg=window_length)
# Print the shape of the STFT result
print("STFT shape:", X.shape)

STFT shape: (129, 115)


In [33]:
len(time)

115

In [34]:
X

array([[-5.25218726e-03+0.00000000e+00j, -1.17549176e-02+0.00000000e+00j,
         2.83644508e-03+0.00000000e+00j, ...,
         2.24710668e-02+0.00000000e+00j,  4.08709108e-03+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [-2.16113397e-03-7.25068759e-03j,  2.87110801e-03+3.55796617e-03j,
        -4.39967140e-03-4.34880163e-04j, ...,
        -1.36689659e-02+6.68728105e-03j,  1.49502530e-03-3.68358778e-03j,
         0.00000000e+00+0.00000000e+00j],
       [ 1.08425446e-02-2.92868741e-04j,  7.47955380e-03+5.34614973e-03j,
         3.73226026e-03+2.06304374e-03j, ...,
         3.27981206e-03-9.34120796e-03j, -2.67251109e-03-2.50668232e-03j,
         0.00000000e+00+0.00000000e+00j],
       ...,
       [-6.17108233e-05+2.13504981e-06j,  3.47889641e-07-3.55724738e-07j,
         1.98882122e-07-3.88802153e-07j, ...,
        -1.98624490e-06-3.51479458e-07j, -1.75824644e-06+8.57762057e-08j,
         0.00000000e+00+0.00000000e+00j],
       [ 6.19766194e-05-1.12116052e-06j, -6.

In [35]:
X.shape


(129, 115)

In [36]:
# function for SNR


def apply_snr(hrir,noise, snr):
    
    duration = 4.5 / 1000  # 4.5 ms

    # Define the original and target sampling rates
    original_fs = 1 / duration  # Sampling rate in Hz
    target_fs = 16000  # Target sampling rate in Hz

    # Calculate the number of samples in the original HRIR
    original_num_samples = len(hrir)

    # Calculate the number of samples in the target HRIR
    target_num_samples = int(original_num_samples * (target_fs / original_fs))

    # Resample the HRIR to the target sampling rate
    signal = resample(hrir, target_num_samples)

    # Load the WAV file
    #sample_rate, signal = wavfile.read(wav_file)

    # Calculate the noise energy
    signal_power = np.mean(signal ** 2)
    
    desired_noise_power = signal_power / (10**(snr / 10))

    # Scale the noise to the desired power
    current_noise_power = np.mean(noise**2)
    scaled_noise = noise * np.sqrt(desired_noise_power / current_noise_power)

    return scaled_noise

In [37]:
def apply_stft_hrir(hrir):
    
    duration = 4.5 / 1000  # 4.5 ms

    # Define the original and target sampling rates
    original_fs = 1 / duration  # Sampling rate in Hz
    target_fs = 16000  # Target sampling rate in Hz

    # Calculate the number of samples in the original HRIR
    original_num_samples = len(hrir)

    # Calculate the number of samples in the target HRIR
    target_num_samples = int(original_num_samples * (target_fs / original_fs))

    # Resample the HRIR to the target sampling rate
    resampled_hrir = resample(hrir, target_num_samples)
    
    #xx=resampled_hrir
    
    
    window_length = 256
    frame_shift = 128

    # Calculate the required padding length
    padding_length = window_length - (14400 % window_length)

    # Add padding to the time series array
    padded_time_series = np.pad(resampled_hrir, (0, padding_length), 'constant')

    # Apply STFT
    frequencies, times, Zxx = stft(padded_time_series, fs=16000, window='hann', nperseg=window_length, noverlap=window_length-frame_shift)

    # Print the shape of the STFT result
#     print("STFT shape:", Zxx.shape)
#     ret = np.arange(128 * 31).reshape(128, 31)
#     ret=Zxx[1:129][:31]
    list=[]
    for i in range(128):
        row = []
        for j in range(31):
            row.append(Zxx[i+1][j])
        list.append(row)
    return list

In [38]:
def apply_stft_noise(hrir,noise,snr):
    xx=apply_snr(hrir,noise,snr)
    
    window_length = 256
    frame_shift = 128
    padding_length = window_length - (14400 % window_length)
    padded_xx = np.pad(xx, (0, padding_length), 'constant')
    frequencies, time, X=stft(padded_xx, fs=16000, window='hann', nperseg=window_length)
    # Print the shape of the STFT result
#     print("STFT shape:", X.shape)
#     ret = np.arange(128 * 31).reshape(128, 31)
#     ret=X[1:129][0:31]
    list=[]
    for i in range(128):
        row = []
        for j in range(31):
            row.append(X[i+1][j])
        list.append(row)
    return list

    

In [39]:
import json

In [40]:
f = open('dictonary_DP_RTF.json')

In [41]:
jk=json.load(f)

In [42]:
type(jk['subject_003']['hrir_l'][0])

list

In [43]:
jk.keys()

dict_keys(['subject_003', 'subject_008', 'subject_009', 'subject_010', 'subject_011', 'subject_012', 'subject_015', 'subject_017', 'subject_018', 'subject_019', 'subject_020', 'subject_021', 'subject_027', 'subject_028', 'subject_033', 'subject_040', 'subject_044', 'subject_048', 'subject_050', 'subject_051', 'subject_058', 'subject_059', 'subject_060', 'subject_061', 'subject_065', 'subject_119', 'subject_124', 'subject_126', 'subject_127', 'subject_131', 'subject_133', 'subject_134', 'subject_135', 'subject_137', 'subject_147', 'subject_148', 'subject_152', 'subject_153', 'subject_154', 'subject_155', 'subject_156', 'subject_158', 'subject_162', 'subject_163', 'subject_165'])

In [44]:
training=['subject_010', 'subject_011', 'subject_012', 'subject_015', 'subject_017', 'subject_018',   'subject_028',  'subject_044', 'subject_048', 'subject_050', 'subject_051', 'subject_058', 'subject_059', 'subject_060',  'subject_124',  'subject_127', 'subject_134', 'subject_135', 'subject_137', 'subject_147', 'subject_148', 'subject_152', 'subject_153', 'subject_154', 'subject_155', 'subject_156', 'subject_158', 'subject_162', 'subject_163', 'subject_165']

In [45]:
validation=['subject_061', 'subject_065', 'subject_119','subject_126', 'subject_131', 'subject_133']

In [46]:
test=['subject_003', 'subject_008', 'subject_009', 'subject_019', 'subject_020', 'subject_021','subject_027','subject_033', 'subject_040']

In [136]:
len(validation)

6

In [181]:
type(jk['subject_028']['hrir_l'][0][0])

list

In [339]:
from collections.abc import Iterable
def get_dimensions(lst):
    dimensions = []
    while isinstance(lst, Iterable):
        dimensions.append(len(lst))
        lst = lst[0] if lst else None
    return dimensions

In [365]:
dc={}


dict

In [186]:
for a in ['subject_003']:
    hrtf_r=apply_stft_hrir(jk[a]['hrir_r'][0][0])[1:][:31]
    hrtf_l=apply_stft_hrir(jk[a]['hrir_l'][0][0])[1:][:31]
    dc[a]=[hrtf_l,hrtf_r]

STFT shape: (129, 115)
STFT shape: (129, 115)


In [366]:
for a in ['subject_003']:
    fin=apply_function_to_2d_list(jk[a]['hrir_l'],jk[a]['hrir_r'])
#     hrtf_l=apply_function_to_2d_list()

#     for i in range(hrtf_r):
#         for j in range (hrtf_r[0]):
            
        
    dc[a]=fin

In [3]:
main_dc={}
t_dc={}
v_dc={}
test_dc={}

In [ ]:

for a in training:
    fin=apply_function_to_2d_list(jk[a]['hrir_l'],jk[a]['hrir_r'])

    t_dc[a]=fin
main_dc['training']=t_dc
for a in validation:
    fin=apply_function_to_2d_list(jk[a]['hrir_l'],jk[a]['hrir_r'])

    v_dc[a]=fin
main_dc['validation']=v_dc
for a in test:
    fin=apply_function_to_2d_list(jk[a]['hrir_l'],jk[a]['hrir_r'])

    test_dc[a]=fin
main_dc['testing']=test_dc

In [4]:
main_dc


{}

In [367]:
print(len(dc['subject_003']))
print(len(dc['subject_003'][0]))
print(len(dc['subject_003'][0][0]))
print(len(dc['subject_003'][0][0][0]))
print(len(dc['subject_003'][0][0][0][0]))
print(type(dc))
print(type(dc['subject_003']))
print(type(dc['subject_003'][0]))
print(type(dc['subject_003'][0][0]))
print(type(dc['subject_003'][0][0][0]))
print(type(dc['subject_003'][0][0][0][0]))

25
50
128
31
2
<class 'dict'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [368]:
get_dimensions(dc['subject_003'])

[25, 50, 128, 31, 2]

In [310]:
# def apply_function_to_2d_list(list):
#     new_list = []
#     for row in list:
#         new_row = [apply_stft_hrir(element) for element in row]
#         new_r=new_row
# #         print(len(new_r[0][0]))
# #         print(len(new_r[0]))
#         new_list.append(new_r)
#     return new_list

In [364]:
#new hrir l r
def apply_function_to_2d_list(list_1,list_2):
    new_list = []
    for i in range(len(list_1)):
        new_row=[]
        for j in range(len(list_1[0])):
            l1=apply_stft_hrir(list_1[i][j])
            n1=apply_stft_noise(list_1[i][j],xx,5)
#             list1=[x + y for x, y in zip(l1,n1)]
            list1=[[l1[i][j] + n1[i][j] for j in range(len(l1[0]))] for i in range(len(l1))]
            l2=apply_stft_hrir(list_2[i][j])
            n2=apply_stft_noise(list_2[i][j],xx,5)
#             list2=[x + y for x, y in zip(l2,n2)]
            list2=[[l2[i][j] + n2[i][j] for j in range(len(l2[0]))] for i in range(len(l2))]
            m = len(list1)
            n = len(list1[0])
            list3 = []
            for ii in range(m):
                row = []
                for jj in range(n):
                    row.append([list1[ii][jj], list2[ii][jj]])
                list3.append(row)
            new_row.append(list3)
            
            
#         new_row = [apply_stft_hrir(element) for element in row]
#         new_r=new_row
#         print(len(new_r[0][0]))
#         print(len(new_r[0]))
        new_list.append(new_row)
    return new_list

In [248]:
import numpy as np


def create_list_of_size_128x31_using_numpy():
 

    list = np.arange(128 * 31).reshape(128, 31)
    return list


# if __name__ == "__main__":
list = create_list_of_size_128x31_using_numpy()
print(list)


[[   0    1    2 ...   28   29   30]
 [  31   32   33 ...   59   60   61]
 [  62   63   64 ...   90   91   92]
 ...
 [3875 3876 3877 ... 3903 3904 3905]
 [3906 3907 3908 ... 3934 3935 3936]
 [3937 3938 3939 ... 3965 3966 3967]]


In [303]:
dc['subject_003'][0][0][0]

[[(-1.9578600636346914e-05+1.629279190708951e-05j),
  (2.5891808196454656e-05+3.901765780912153e-05j),
  (6.977939107625798e-05-4.635422107810922e-06j),
  (0.0002489506021714121-0.00013451817451689825j),
  (0.00048619537909361634-0.0004078759892241334j),
  (0.002300410946886451+0.0012601937535946912j),
  (-0.011987047042748045-0.0014686704721464938j),
  (0.007735973975933525-0.00036055722067695995j),
  (0.008006282337166291+0.004129078231314945j),
  (0.0021538610384644248-0.008911638983784321j),
  (-0.011482587152122536+0.006855686118048106j),
  (0.00858477132476632+0.005952660089191099j),
  (0.02289523997391413+0.07012732778013445j),
  (-0.6550208547714093+0.13280780722221044j),
  (0.3157087761773803-0.3891730212833591j),
  (0.13303117872912573+0.13687796511296238j),
  (0.17911968411450724-0.1476850966866813j),
  (0.24132150267287006+0.3337954481043147j),
  (-0.5416029839470046-0.08571468729043022j),
  (0.26962477693262055-0.09619968083506271j),
  (0.06271801612010244-0.04243914731072

In [259]:
len(dc['subject_003'][0][0][0][0])

115

In [191]:
dc['subject_003']

[[[(-1.9578600636346914e-05+1.629279190708951e-05j),
   (2.5891808196454656e-05+3.901765780912153e-05j),
   (6.977939107625798e-05-4.635422107810922e-06j),
   (0.0002489506021714121-0.00013451817451689825j),
   (0.00048619537909361634-0.0004078759892241334j),
   (0.002300410946886451+0.0012601937535946912j),
   (-0.011987047042748045-0.0014686704721464938j),
   (0.007735973975933525-0.00036055722067695995j),
   (0.008006282337166291+0.004129078231314945j),
   (0.0021538610384644248-0.008911638983784321j),
   (-0.011482587152122536+0.006855686118048106j),
   (0.00858477132476632+0.005952660089191099j),
   (0.02289523997391413+0.07012732778013445j),
   (-0.6550208547714093+0.13280780722221044j),
   (0.3157087761773803-0.3891730212833591j),
   (0.13303117872912573+0.13687796511296238j),
   (0.17911968411450724-0.1476850966866813j),
   (0.24132150267287006+0.3337954481043147j),
   (-0.5416029839470046-0.08571468729043022j),
   (0.26962477693262055-0.09619968083506271j),
   (0.0627180161201

In [177]:

def array_to_list(array):


    list = []
    for item in array:
        list.append(item)
        
    return list



array = [1, 2, 3, 4, 5]
print(array)
list = array_to_list(array)
print(list)

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]


In [146]:
jk["subject_003"].keys()

dict_keys(['OnR', 'OnL', 'ITD', 'hrir_r', 'hrir_l', 'name'])

In [139]:
#building CNN
# Assuming the input shape for each frame is FxNx2
F = 128  # Number of frequency bins
N = 31  # Number of time frames

In [140]:
# Input layers
input_mag = Input(shape=(F, N, 2))  # Magnitude spectrogram input
input_phase = Input(shape=(F, N, 2))  # Phase spectrogram input